In [3]:
import time

from data.swarmset import SwarmDataset, DataBuilder

baseline_data = DataBuilder("data/full", steps=1200, agents=24)
baseline_data.create()
baseline_data.evolution.close()

Exception: Requested to build new dataset in folder that contains items

# Embed into Latent Space

In [ ]:
from networks.embedding import NoveltyEmbedding
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
network = NoveltyEmbedding().to(device)
print(network)

In [1]:
def network_from_numpy(network, anchor_img, pos_img, neg_img):
    anchor_input = torch.from_numpy(anchor_img).to(device).float()
    pos_input = torch.from_numpy(pos_img).to(device).float()
    neg_input = torch.from_numpy(neg_img).to(device).float()

    anchor_out = network(anchor_input.unsqueeze(0))
    pos_out = network(pos_input.unsqueeze(0))
    neg_out = network(neg_input.unsqueeze(0))
    return anchor_out, pos_out, neg_out

def batch_network_from_numpy(network, anchor_list, pos_list, neg_list):
    anchor_input = torch.from_numpy(anchor_list).to(device).float()
    pos_input = torch.from_numpy(pos_list).to(device).float()
    neg_input = torch.from_numpy(neg_list).to(device).float()

    anchor_out = network(anchor_input)
    pos_out = network(pos_input)
    neg_out = network(neg_input)
    return anchor_out, pos_out, neg_out

def combine_hierarchy_samples(class_a, class_b):
    out = []
    for ancA, posA, negA in class_a:
        for ancB, posB, negB in class_b:
            if posB != posA:
                out.append((posA, posB, negA))
                out.append((posB, posA, negB))
    return out

In [2]:
# Human in the Loop
from sklearn.manifold import TSNE
import pygame
import torch
from sklearn_extra.cluster import KMedoids
from ui.class_similarity import SimilarityGUI
from networks.archive import DataAggregationArchive

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

keys={
    pygame.K_KP_PLUS : -1,
    pygame.K_KP_PERIOD: -1,
    pygame.K_KP_0 : 0,
    pygame.K_KP_1 : 1,
    pygame.K_KP_2 : 2,
    pygame.K_KP_3 : 3,
    pygame.K_KP_4 : 4,
}

gui_labels = {
    -1 : "Random",
    0 : "0",
    1 : "1",
    2 : "2",
    3 : "3",
    4 : "4",
}

CLUSTERS = 8
def humanInput(anchor_dataset, network, optim, loss_fn, data_archive, random_indices, stop_at=None):
    # Begin by clustering all known embeddings into n classes.
    # Sample from these clusters and use the medoids as the anchors to hopefully scoop up hard samples.
    network.eval()
    archive = NoveltyArchive()
    for i, (anchor_encoding, genome, _, _, _, _, _) in enumerate(anchor_dataset):
        if stop_at and i > stop_at:
            break
        if i in random_indices.archive:
            archive.addToArchive(vec=np.array([-1] * 15), genome=genome)
        else:
            anchor_encoding = torch.from_numpy(anchor_encoding).to(device).float()
            embedding = network(anchor_encoding.unsqueeze(0)).squeeze(0).cpu().detach().numpy()
            archive.addToArchive(vec=embedding, genome=genome)

    print("Query + Cluter at size: ", len(archive.archive))

    kmedoids = KMedoids(n_clusters=CLUSTERS, random_state=0).fit(archive.archive)
    labels = kmedoids.labels_
    medoids = kmedoids.medoid_indices_

    network.train()
    user_help_cases, avg_loss, total_attempts = 0, 0, 1
    skip_clusters = []
    cluster_sampling = {}
    for cluster_class in range(CLUSTERS + 1):

        # Run Triplet Query on the Medoids themselves
        if cluster_class == 0:
            samples = [medoids[med] for med in range(0, len(medoids))]
            subject_images = [anchor_dataset[sampled_class][0] for sampled_class in samples]
            ui_input = SimilarityGUI(None, subject_images, keystrokes=keys, labels=gui_labels)
            ui_input.run()
            user_responses = np.array(ui_input.assignment)

            # Assign all the skipped classes
            for i, res in enumerate(user_responses):
                if res < 0:
                    skip_clusters.append(i)
                    random_indices.append_scalars([medoids[i]])

            # Run triplet loss on medoids.
            for i in range(5):
                same_class = np.where(user_responses == i)[0]
                different_class = np.where(user_responses != i)[0]
                if len(same_class) < 2:
                    continue
                for j in range(len(same_class)):
                    for k in range(i + 1, len(same_class)):
                        for l in range(len(different_class)):
                            anchor_image = anchor_dataset[medoids[j]][0]
                            pos_image = anchor_dataset[medoids[k]][0]
                            neg_image = anchor_dataset[medoids[l]][0]

                            optimizer.zero_grad()
                            anchor_out, pos_out, neg_out = network_from_numpy(network, anchor_image, pos_image, neg_image)

                            loss = loss_fn(anchor_out, pos_out, neg_out)
                            avg_loss += loss.item()
                            total_attempts += 1
                            print("User loss: ", loss)
                            if loss.item() > 0:
                                loss.backward()
                                optim.step()
                                user_help_cases += 1
                                print("BACKPROP!")

                            # Add to User Collected Archive
                            data_archive.append(medoids[j], medoids[k], medoids[l])
                            for random in skip_clusters:
                                data_archive.append(medoids[j], medoids[k], medoids[random])

        else:
            cluster_class -= 1
            if cluster_class in skip_clusters:
                continue

            # Initialize Cluster Sampling
            cluster_sampling[cluster_class] = []

            examples = np.where(labels == cluster_class)[0]
            samples = np.random.choice(examples, min(10, len(examples)), False)

            anchor_image = anchor_dataset[medoids[cluster_class]][0]
            subject_images = [anchor_dataset[sampled_class][0] for sampled_class in samples]
            ui_input = SimilarityGUI(anchor_image, subject_images)
            ui_input.run()
            user_responses = np.array(ui_input.assignment)

            different_class = np.where(user_responses == 1)[0]
            same_class = np.where(user_responses == 0)[0]
            for i in range(len(different_class)):
                for j in range(i, len(same_class)):
                    pos_index = same_class[j]
                    neg_index = different_class[i]
                    anchor_image = anchor_dataset[medoids[cluster_class]][0]
                    pos_image = anchor_dataset[samples[pos_index]][0]
                    neg_image = anchor_dataset[samples[neg_index]][0]

                    optimizer.zero_grad()
                    anchor_out, pos_out, neg_out = network_from_numpy(network, anchor_image, pos_image, neg_image)

                    loss = loss_fn(anchor_out, pos_out, neg_out)
                    avg_loss += loss.item()
                    total_attempts += 1
                    # print("User loss: ", loss)
                    if loss.item() > 0:
                        loss.backward()
                        optim.step()
                        user_help_cases += 1
                        # print("BACKPROP!")

                    # Add to User Collected Archive
                    data_archive.append(medoids[cluster_class], samples[pos_index], samples[neg_index])
                    cluster_sampling[cluster_class].append((medoids[cluster_class], samples[pos_index], samples[neg_index]))


    # Run Hierarchical Comparison in the cluster elements.
    for i in range(5):
        same_class = np.where(user_responses == i)[0]
        for j in range(len(same_class)):
            for k in range(i + 1, len(same_class)):
                if j in cluster_sampling and k in cluster_sampling:
                    combined_samples = combine_hierarchy_samples(cluster_sampling[j], cluster_sampling[k])
                    for anc, pos, neg in combined_samples:
                        data_archive.append(anc, pos, neg)

    return user_help_cases, avg_loss / total_attempts

pygame 2.1.2 (SDL 2.0.16, Python 3.10.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
import torch
from data.swarmset import SwarmDataset, DataBuilder
from networks.embedding import NoveltyEmbedding
from generation.evolution import ModifiedHaltingEvolution
from NovelSwarmBehavior.novel_swarms.novelty.NoveltyArchive import NoveltyArchive
from NovelSwarmBehavior.novel_swarms.config.ResultsConfig import ResultsConfig
from NovelSwarmBehavior.novel_swarms.results.results import main as results
from NovelSwarmBehavior.novel_swarms.config.defaults import ConfigurationDefaults
from matplotlib import pyplot as plot
import numpy as np

TRAIN = True
CLUSTER_AND_DISPLAY = True
WRITE_OUT = False
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

network = NoveltyEmbedding().to(device)
anchor_dataset = SwarmDataset("data/full", rank=0)
evolution, screen = ModifiedHaltingEvolution.defaultEvolver(steps=500, n_agents=10)

optimizer = torch.optim.Adam(network.parameters(), lr=1e-3)

# Margin was 10
loss_fn = torch.nn.TripletMarginLoss(margin=15)

EPOCHS = 50
self_l_hist = []
random_indices = []
human_l_hist = [0.0 for _ in range(EPOCHS)]
HIL_archive = DataAggregationArchive()
random_archive = DataAggregationArchive(scalar=True)
data_set_size = 1000
EPSILON = 0.5

if TRAIN:
    STOP_FLAG = False
    for epoch in range(EPOCHS):
        if STOP_FLAG:
            break

        repeats = False if len(self_l_hist) < 3 else (self_l_hist[-3] < sum(self_l_hist[-3:]) / 3)
        if len(HIL_archive) == 0 or (self_l_hist and self_l_hist[-1] < EPSILON) or repeats:
            print("HIL TIME!")
            improvements, human_loss = humanInput(anchor_dataset, network, optimizer, loss_fn, HIL_archive, random_archive, stop_at=data_set_size)
            print(f"Improvement Count: {improvements}, loss: {human_loss}")
            human_l_hist[epoch] = human_loss

            HIL_archive.save_to_file("data/queries/triplets2.csv")
            random_archive.save_to_file("data/queries/random2.csv")

        data_set_size += 100

        loss_sum = 0
        total_loss = 0
        # Train on known archive
        print(f"User Data at size: {len(HIL_archive)}")

        # Train on past user information]
        BATCH_SIZE = 20
        anchor_list, pos_list, neg_list = None, None, None
        for i, (anchor, pos, neg) in enumerate(HIL_archive):
            anchor_encoding = np.expand_dims(anchor_dataset[anchor][0], axis=0)
            similar_encoding = np.expand_dims(anchor_dataset[pos][0], axis=0)
            anti_encoding = np.expand_dims(anchor_dataset[neg][0], axis=0)

            if anchor_list is None:
                anchor_list = np.array([anchor_encoding])
                pos_list = np.array([similar_encoding])
                neg_list = np.array([anti_encoding])

            else:
                anchor_list = np.concatenate((anchor_list, np.array([anchor_encoding])))
                pos_list = np.concatenate((pos_list, np.array([similar_encoding])))
                neg_list = np.concatenate((neg_list, np.array([anti_encoding])))

            if len(anchor_list) >= BATCH_SIZE:
                optimizer.zero_grad()
                anchor_out, pos_out, neg_out = batch_network_from_numpy(network, anchor_list, pos_list, neg_list)
                loss = loss_fn(anchor_out, pos_out, neg_out)
                loss_sum += loss.item()
                if loss.item() > 0:
                    loss.backward()
                    optimizer.step()
                total_loss += 1
                anchor_list, pos_list, neg_list = None, None, None

            if i > 0 and i % 500 == 0:
                print(f"Epoch Progress: {(i*100) / len(HIL_archive)}%, Immediate Loss: {loss.item()}")

        print(f"Average Loss: {loss_sum / (total_loss + 1)}")
        self_l_hist.append(loss_sum / (total_loss + 1))

        if len(self_l_hist) > 3 and self_l_hist[-1] == self_l_hist[-2] and self_l_hist[-2] == self_l_hist[-3]:
            loss_sum = 0

        print(f"Epoch: {epoch}")

evolution.close()

User Data at size: 1
Average Loss: 0.0
Epoch: 0
HIL TIME!
Query + Cluter at size:  1101
Improvement Count: 0, loss: 0.0
User Data at size: 1
Average Loss: 0.0
Epoch: 1
HIL TIME!
Query + Cluter at size:  1201


KeyboardInterrupt: 

In [ ]:
# Save Model
network.save_model()

In [13]:
import time
import torch
from data.swarmset import SwarmDataset, DataBuilder
from networks.embedding import NoveltyEmbedding
from NovelSwarmBehavior.novel_swarms.novelty.NoveltyArchive import NoveltyArchive
from NovelSwarmBehavior.novel_swarms.config.ResultsConfig import ResultsConfig
from NovelSwarmBehavior.novel_swarms.results.results import main as results
from NovelSwarmBehavior.novel_swarms.config.defaults import ConfigurationDefaults
from data.swarmset import SwarmDataset, DataBuilder

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
network = NoveltyEmbedding().to(device)
network.load_model("testA")
anchor_dataset = SwarmDataset("data/full", rank=0)

WRITE_OUT = True
if WRITE_OUT:
    network.eval()
    test_archive = NoveltyArchive()
    for i, (anchor_encoding, genome, _, _, _, _, _) in enumerate(anchor_dataset):
        if i > 5000:
            break
        anchor_encoding = torch.from_numpy(anchor_encoding).to(device).float()
        embedding = network(anchor_encoding.unsqueeze(0)).squeeze(0).cpu().detach().numpy()
        test_archive.addToArchive(embedding, genome)

In [14]:
from ui.clustering_gui import ClusteringGUI
import pygame

agent_config = ConfigurationDefaults.DIFF_DRIVE_AGENT
world_config = ConfigurationDefaults.RECTANGULAR_WORLD
world_config.addAgentConfig(agent_config)
config = ResultsConfig(archive=test_archive, k_clusters=8, world_config=world_config, tsne_perplexity=1, tsne_early_exaggeration=1, skip_tsne=False)
gui = ClusteringGUI(config)
gui.displayGUI()
# results(config)
pygame.quit()

/home/connor/Desktop/research/SwarmNoveltyNetwork/.env/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:996: FutureWarning: The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.
  warnings.warn(


KeyboardInterrupt: 

# Clustering + Analysis

In [10]:
from ui.clustering_gui import ClusteringGUI

# Cluster over saved behaviors
archive = NoveltyArchive()
for i, (_, genome, behavior, _, _, _, _) in enumerate(anchor_dataset):
    archive.addToArchive(vec=behavior, genome=genome)

agent_config = ConfigurationDefaults.DIFF_DRIVE_AGENT
world_config = ConfigurationDefaults.RECTANGULAR_WORLD
world_config.addAgentConfig(agent_config)
config = ResultsConfig(archive=archive, k_clusters=7, world_config=world_config, tsne_perplexity=20, tsne_early_exaggeration=2, skip_tsne=False)

gui = ClusteringGUI(config)
gui.displayGUI()

/home/connor/Desktop/research/SwarmNoveltyNetwork/.env/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:996: FutureWarning: The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.
  warnings.warn(


In [ ]:
import matplotlib.pyplot as plt

plt.plot(human_l_hist, "b", label='Human Loss')
plt.plot(self_l_hist, "r", label='Self Loss')
plt.ylabel("Loss")
plt.xlabel("Time (Epochs)")
plt.legend()